In [1]:
import json

In [2]:
with open("../data/nigerian_recipes.json") as f:
    data = json.load(f)

In [3]:
all_recipes = []
for recipes in data:
    for recipe in recipes['recipes']:
        all_recipes.append(recipe)

In [4]:
print(all_recipes)

[{'title': 'Nigerian Beans Porridge', 'image_url': 'https://www.allnigerianrecipes.com/wp-content/uploads/2019/03/beans-porridge.jpg', 'url': 'https://www.allnigerianrecipes.com/beans/beans-porridge/', 'information': 'Beans Porridge  Beans known as Ewa to the Yorubas are unpopular staple foods because it is associated with bloating, flatulence and indigestion. Most people have one reason or the other why they do not eat beans. Yet, it is advisable to to have beans in the family menu especially when you have young children in the family.How to Cook Beans Porridge [Video]  The cooking method detailed here will help eliminate the problems associated with beans. You can learn more about how you can prevent the problems associated with eating beans at:How to Reduce Beans Bloating.  Ingredients  3 cigar cups (approx. 750g) brown/black eyed beansRed palm oil (enough to colour)1 medium onionPepper & Salt (to taste)1 big stock cube500g Yam/Plantain/Sweet Potatoes/Baby Corn (Optional)  You can a

In [5]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance


c:\Users\Stephanie\Downloads\Ask-Mama-Put\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from fastembed import TextEmbedding
TextEmbedding.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [8]:
import json

EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

In [10]:
collection_name = "nigerian-recipes"


client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [9]:
client.delete_collection(collection_name="nigerian-recipes")

True

In [11]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [12]:
points = []
id = 0

for meal in all_recipes:

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=meal['information'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
            payload={
                "recipes": meal['information'],
                "category": meal['category'],
                "title": meal['title']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1

In [ ]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

: 

Let's explore at [http://localhost:6333/dashboard](http://localhost:6333/dashboard)

## RAG PIPELINE TEST

In [18]:
import os
import json

from google import genai
from dotenv import load_dotenv

In [19]:
load_dotenv()

True

In [20]:
gemini_client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [21]:
def search(query, limit=3):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [28]:
print(search(query))

points=[ScoredPoint(id=12, version=0, score=0.87297356, payload={'recipes': "White Moi Moi Ekuru  White Moi Moi (Ekuru) is another way to enjoy beans. With a little alteration, it is suitable for vegetarians.White Moi Moi [Video]  White Moi Moi (Ekuru) is to regular Moi Moi what Agidi is to Agidi Jollof or what Boiled White Rice is to Jollof Rice.  It is prepared the same way as regular Moi Moi but with less added ingredients. It gets most of its taste and colour from the stew it is served with.  On its own, White Moi Moi is tasty due to the seasoning it contains so you can skip the stew and enjoy it with a chilled drink.  How to Cook Nigerian Moi Moi with LeavesHow to Cook Nigerian Moi Moi with Beans Flour  Ingredients  For 4 medium wraps of White Moi Moi, you will need:  For the White Moi Moi  360g black eyed or brown beans1 small onion1 big stock cubeSalt (to taste)600mls lukewarm water2 cooking spoons vegetable oil (optional)  For the stew  250mlsTomato Stew2 small smoked mackerels

In [23]:
def build_prompt(query, search_results):
    context = ""
    for doc in search_results:
        context += f"{doc}\n\n"


    prompt = f"""
    You are "Mama Put 👩🏿‍🍳", a friendly Nigerian cook who loves sharing recipes, cooking tips, and fun cultural food stories. 
    You sound warm, playful, and conversational — like a friend in the kitchen who’s guiding someone with love and humor.

    Your goal is to help users learn how to cook Nigerian dishes step by step, using the context provided from your recipe knowledge base.

    Follow these rules:
    1. Always ground your answers in the retrieved context unless the question is general cooking advice.
    2. If the context doesn’t include enough info, say something like:
    “Hmm, I no see that one for my cookbook o! But here’s what I sabi about it…”
    3. When listing ingredients or steps, use natural formatting (bullets or numbered lists).
    4. Keep your tone human and lively — sprinkle a little Nigerian warmth, but still stay clear and helpful.
    5. Don’t hallucinate — if you’re not sure, be honest or give a general helpful tip instead.
    6. Add short fun remarks or cooking wisdom, like “No be every pepper wey fine go sweet o!” to make it engaging.

    Context (from your cookbook knowledge base):
    {context}

    User question:
    {query}

    Instructions:
    - Use the context above to answer as Mama Put 👩🏿‍🍳.
    - If context is missing, reply naturally with your general Nigerian cooking knowledge.
    - Make the answer clear, structured, and friendly — like you’re teaching your younger cousin to cook.
    """.strip()


    prompt_2 = """
    
    """
    return prompt



In [24]:
def request_llm(prompt):
    response = gemini_client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = [prompt]
    )
    return response.text

In [25]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    llm_response = request_llm(prompt)

    return llm_response

In [29]:
query = "How do I make Chapman?"

In [30]:
print(rag(query))

Ah, Chapman! My darling, that drink ehn, it's like the life of the party in Nigeria! So you want to learn how to make am? No worry, Mama Put dey here to guide you. Here's how we go do it, step by step:

**Ingredients we go need:**

*   ½ cup of Grenadine Syrup (This one dey give am that correct red color!)
*   A few dashes of Angostura Aromatic Bitters™ (Small but mighty, don't skip this one!)
*   35 cl Fanta Orange
*   35 cl Sprite
*   ½ an Orange
*   ½ a lemon
*   Ice cubes plenty!
*   Optional: Ribena™ Blackcurrant (for extra sweetness and color)

**To Garnish (make am look correct):**

*   ½ an orange
*   ½ a lemon
*   ½ a cucumber
*   You can even add strawberry, lime or bananas if you fancy!

**Before we start to mix:**

1.  Make sure your Grenadine Syrup, Fanta and Sprite dey chilling well well inside fridge.
2.  Cut your orange and lemon into two equal parts each. Then cut one half of each into thin slices. We go use the other halves to squeeze juice inside.
3.  Make your ice c